In [1]:
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import pickle

In [2]:
class TrainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [3]:
class TestData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [4]:
class BinaryClassification(torch.nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        # Number of input features is 12.
        self.layer_1 = torch.nn.Linear(192, 64) 
        self.layer_2 = torch.nn.Linear(64, 64)
        self.layer_out = torch.nn.Linear(64, 1) 
        
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p=0.1)
        self.batchnorm1 = torch.nn.BatchNorm1d(64)
        self.batchnorm2 = torch.nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [5]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [6]:
EPOCHS = 500
BATCH_SIZE = 10
LEARNING_RATE = 0.001
base_path = "/content/drive/MyDrive/Okul/İTÜ/Bilgisayar Mühendisliği Yüksek Lisans/Yüksek Lisans/YL Tez/Development/UNSW-NB15"
X_train = np.genfromtxt(f"{base_path}/dataset/30-sampled-x-train.csv", delimiter=',')
y_train = np.genfromtxt(f"{base_path}/dataset/30-sampled-y-train.csv", delimiter=',')
X_test = np.genfromtxt(f"{base_path}/dataset/30-sampled-x-test.csv", delimiter=',')
y_test = np.genfromtxt(f"{base_path}/dataset/30-sampled-y-test.csv", delimiter=',')

In [8]:
for rate in [30]:
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  y_train = y_train.astype("int64")
  train_data = TrainData(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
  train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  model = BinaryClassification()
  model.to(device)
  criterion = torch.nn.BCEWithLogitsLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
  model.train()
  for e in range(1, EPOCHS+1):
      epoch_loss = 0
      epoch_acc = 0
      for X_batch, y_batch in train_loader:
          X_batch, y_batch = X_batch.to(device), y_batch.to(device)
          optimizer.zero_grad()
          
          y_pred = model(X_batch)
          
          loss = criterion(y_pred, y_batch.unsqueeze(1))
          acc = binary_acc(y_pred, y_batch.unsqueeze(1))
          
          loss.backward()
          optimizer.step()
          
          epoch_loss += loss.item()
          epoch_acc += acc.item()
          

      print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

Epoch 001: | Loss: 0.05334 | Acc: 98.475
Epoch 002: | Loss: 0.02594 | Acc: 99.154
Epoch 003: | Loss: 0.02280 | Acc: 99.222
Epoch 004: | Loss: 0.02003 | Acc: 99.283
Epoch 005: | Loss: 0.01931 | Acc: 99.351
Epoch 006: | Loss: 0.01833 | Acc: 99.354
Epoch 007: | Loss: 0.01714 | Acc: 99.400
Epoch 008: | Loss: 0.01826 | Acc: 99.354
Epoch 009: | Loss: 0.01710 | Acc: 99.452
Epoch 010: | Loss: 0.01803 | Acc: 99.354
Epoch 011: | Loss: 0.01528 | Acc: 99.450
Epoch 012: | Loss: 0.01626 | Acc: 99.436
Epoch 013: | Loss: 0.01539 | Acc: 99.463
Epoch 014: | Loss: 0.01496 | Acc: 99.491
Epoch 015: | Loss: 0.01642 | Acc: 99.441
Epoch 016: | Loss: 0.01443 | Acc: 99.452
Epoch 017: | Loss: 0.01521 | Acc: 99.472
Epoch 018: | Loss: 0.01452 | Acc: 99.496
Epoch 019: | Loss: 0.01478 | Acc: 99.518
Epoch 020: | Loss: 0.01486 | Acc: 99.496
Epoch 021: | Loss: 0.01366 | Acc: 99.461
Epoch 022: | Loss: 0.01416 | Acc: 99.469
Epoch 023: | Loss: 0.01525 | Acc: 99.463
Epoch 024: | Loss: 0.01511 | Acc: 99.504
Epoch 025: | Los

In [9]:
X_test_df = pd.DataFrame(X_test)
X_test_df.to_csv(f"/content/drive/MyDrive/Okul/İTÜ/Bilgisayar Mühendisliği Yüksek Lisans/Yüksek Lisans/YL Tez/Development/UNSW-NB15/dataset/{rate}-rnn-x_test.csv")
y_test_df = pd.DataFrame(y_test)
y_test_df.to_csv(f"/content/drive/MyDrive/Okul/İTÜ/Bilgisayar Mühendisliği Yüksek Lisans/Yüksek Lisans/YL Tez/Development/UNSW-NB15/dataset/{rate}-rnn-y_test.csv")
pickle.dump(model, open(f"/content/drive/MyDrive/Okul/İTÜ/Bilgisayar Mühendisliği Yüksek Lisans/Yüksek Lisans/YL Tez/Development/UNSW-NB15/model/{rate}-rnn-model.pkl", 'wb'))

In [10]:
rate = 30
X_test = pd.read_csv(f"/content/drive/MyDrive/Okul/İTÜ/Bilgisayar Mühendisliği Yüksek Lisans/Yüksek Lisans/YL Tez/Development/UNSW-NB15/dataset/{rate}-rnn-x_test.csv", index_col=0)
y_test = pd.read_csv(f"/content/drive/MyDrive/Okul/İTÜ/Bilgisayar Mühendisliği Yüksek Lisans/Yüksek Lisans/YL Tez/Development/UNSW-NB15/dataset/{rate}-rnn-y_test.csv", index_col=0)
test_data = TestData(torch.FloatTensor(X_test.to_numpy()))
test_loader = DataLoader(dataset=test_data, batch_size=1)

y_pred_list = []
model = pickle.load(open(f"/content/drive/MyDrive/Okul/İTÜ/Bilgisayar Mühendisliği Yüksek Lisans/Yüksek Lisans/YL Tez/Development/UNSW-NB15/model/{rate}-rnn-model.pkl", 'rb'))
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
print(confusion_matrix(y_test, y_pred_list))
print(classification_report(y_test, y_pred_list))

[[12562    20]
 [  103  5304]]
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00     12582
         1.0       1.00      0.98      0.99      5407

    accuracy                           0.99     17989
   macro avg       0.99      0.99      0.99     17989
weighted avg       0.99      0.99      0.99     17989



In [11]:
np.savetxt(f"{base_path}/model/rnn-y-pred.csv", y_pred_list, delimiter=",")